This is me trying to integrate all of the things I've learned: 1. SVM, SVC, torch, torch, pandas, numpy

The objective is to build a SVM algorithm that takes the data set un compute using SVM kernel trick to find the best hyperplain.

# Importing and Reading the data

## Importing

In [42]:
import torch
import pandas as pd
import numpy as np
import file_operations
import os

## Reading

In [43]:
current_dir = os.getcwd()
print(current_dir)
paths = file_operations.create_project_path("Project 6 SVM")
file_name = 'breast-cancer-wisconsin.data'
file_path = os.path.join(paths["data_dir"], file_name)
df = pd.read_csv(file_path)
display(df)
display(df.info())

d:\Important Files\Repositories\Machine-Learning-Project-related\Project 6 SVM
Project directory created under 'd:\Important Files\Repositories\Machine-Learning-Project-related\Project 6 SVM'
Project data directory created under 'd:\Important Files\Repositories\Machine-Learning-Project-related\Project 6 SVM\Data'
Project output directory created under 'd:\Important Files\Repositories\Machine-Learning-Project-related\Project 6 SVM\Output'


,id,clump_thickness,unif_cell_size,unif_cell_shape,marg_adhesion,single_spith_cell_size,bare_nuclei,bland_chrom,norm_nucleoli,mitoses,class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...
694,776715,3,1,1,1,3,2,1,1,1,2
695,841769,2,1,1,1,2,1,1,1,1,2
696,888820,5,10,10,3,7,3,8,10,2,4
697,897471,4,8,6,4,3,4,10,6,1,4


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   id                      699 non-null    int64 
 1   clump_thickness         699 non-null    int64 
 2   unif_cell_size          699 non-null    int64 
 3   unif_cell_shape         699 non-null    int64 
 4   marg_adhesion           699 non-null    int64 
 5   single_spith_cell_size  699 non-null    int64 
 6   bare_nuclei             699 non-null    object
 7   bland_chrom             699 non-null    int64 
 8   norm_nucleoli           699 non-null    int64 
 9   mitoses                 699 non-null    int64 
 10  class                   699 non-null    int64 
dtypes: int64(10), object(1)
memory usage: 60.2+ KB


None

# Dataframe preprocessing

In [44]:
df.replace('?', np.nan, inplace = True)
df.dropna(inplace = True)
df.drop(['id'], axis = 1, inplace = True)

In [48]:
from sklearn import model_selection, svm
X = np.array(df.drop(['class'], axis = 1)).astype('float64')
y = np.array(df['class']).astype('float64')

# Getting the scikit learn version of accuracy.

In [49]:
%%time
sk_accuracy_sum = 0
count = 0
for count in range(0, 10000):
    X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size = 0.2)
    clf = svm.SVC()
    clf.fit(X_train, y_train)
    accuracy = clf.score(X_test, y_test)
    sk_accuracy_sum += accuracy
    count += 1

print(count)
print(sk_accuracy_sum / count)

10000
0.970024817518212
CPU times: total: 35.8 s
Wall time: 35.8 s


# Creating My Own GPU accelerated version(Binary Classification):

## Thoughts:
1. Checking the dimensions

In [50]:
print(type(X))
print(X.shape)

<class 'numpy.ndarray'>
(683, 9)


2. The feature is 9 dimensional, so the hyper plain best seperating hyperplain should be 8 dimensional.
3. transfer numpy array into tensor stored on gpu.
4. Create a randomly shuffle function to shuffle the rows but not columns of X.
5. Create a SVC weights tensor for later train to train on. 
    The objective is to find the best weights, because SVC in this case is a normal equation of 8 dimensions, we need to find the coefficients for this equation. when found, the equation is the best seperating huperplain.
6. Randomize weights for the SVC
7. A function to find out a single n dimentional point to a (n-1) dimentional span's shortest vector, then multiply that vector with a classifier vector to determine wheather or not that they are reversed vectors with each other.
    This will help determine wheather a dot is on the best seperating heperplain's "positive side" or "negative side".

    Note that the first vector will be considered the "class 1" vector, meaning that it's relation to the best seperating hyperplain will be considered "Class 1"

    For example:
        if a dot is [1, 2], the line best seperating hyperplain is y = x, then the shortest vector from the dot to the hyper plain is [1.5, 1.5] - [1, 2] = [0.5, -0.5]
        the the direction of this vector [0.5, -0.5] will be considered "class 1". The vectors opposed to this vector will be considered "class 2"

## 3. Tranfer numpy into tensor stored on gpu.

### Checking availability

In [51]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Current device: {device.capitalize()}.")

Current device: Cuda.


### Transformation.

In [56]:
X_gpu = torch.tensor(X, device = device)
y_gpu = torch.tensor(y, device = device)
print(X_gpu.shape)
print(y_gpu.shape)

torch.Size([683, 9])
torch.Size([683])


## 4. Shuffle the tensors' row vectors.

In [ ]:
indices = torch.randperm(X_gpu.shape[0])
X_gpu = X_gpu[indices]

torch.Size([683, 9])

## 5. Find soft margin and SVC between each features.(Handling outliers and misclassification)

Equations for the SVC:

`Ax_1 + Bx_2 + Cx_3 + ... + Hx_8 + b = 0`

`W = [A, B, C, D, E, F, G, H]`

`X = [x_1, x_2, x_3, x_4, x_5, x_6, x_7, x_8]`

`b` is a constant

### Create a random 

In [141]:
# print(X.shape)
# (683, 9)
# Randomizing a weight matrix to adjust.
SVC_weights = torch.rand(1, X.shape[1] - 1, dtype = torch.float64, device = device)
SVC_bias = torch.tensor(0.0, dtype=torch.float64, device=device)
print(SVC_weights.shape)
print(SVC_weights)
print(SVC_weights.dtype)
print(SVC_bias)

torch.Size([1, 8])
tensor([[0.8905, 0.5573, 0.9193, 0.3907, 0.9963, 0.7898, 0.4790, 0.8196]],
       device='cuda:0', dtype=torch.float64)
torch.float64
tensor(0., device='cuda:0', dtype=torch.float64)


### Function to culculate the n dimensional direction vector.

In [ ]:
def cal_n_point_dir_vec(n_point, hyperplain_weights, hyperplain_bias):
    # n point should be n dimensional row vector
    # hyperplain_weights shoule be a (n - 1) row vector(row vector: 1 by n)
    # hyperplain_bias should be a float 64 number
    
    # Check input valitility function HERE:
    
    